## Подготовка данных
Тут собирается очень сложно. Должен быть один файл с данными по клиенту и флагом дефолта по нему

In [ ]:
import pandas as pd
igor_path='C:\\Временные файлы\\strategy_full.xlsx'

full_data=pd.read_excel(igor_path)

# Анализ данных

In [ ]:
%run -i datapy/analyze/recomendation.py

server=recommender()

## Поиск однофакторных правил

In [ ]:
#Набор полей, по которым ищутся правила, передаётся клиентом
columns=['ClientKidsNumber','ClientAge','CalculatedMaxLoanAmountCS','LoanYearRate']
#Поле - флаг дефолта, передаётся клиентом
default_column='target_90p_12mob_objective'
result=server.RecommendRules(full_data, columns, default_column)
result

#### Пример вывода данных по предлагаемым правилам пользователю

In [ ]:
#пример обработк полученных правил
def RuleToText(rule):
    column_name=rule[0]
    thr=rule[1]
    trend=rule[2]
    dr_delta=rule[3]
    if trend is None:
        return "Для поля {} не было найдено подходящих правил".format(column_name)
    elif trend:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} выше {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))
    else:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} ниже {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))
    
for rule in result:
    print(RuleToText(rule))

## Поиск многофакторных правил

In [ ]:
%run -i datapy/analyze/recomendation.py
server=recommender()

#Набор полей, по которым ищутся правила, передаётся клиентом
columns=['ClientKidsNumber','ClientAge','CalculatedMaxLoanAmountCS']
#Поле - флаг дефолта, передаётся клиентом
default_column='target_90p_12mob_objective'
best_multirule, dr_delta=server.RecommendMultiRules(full_data, columns, default_column)

#### Пример вывода данных по предлагаемым мульти-правилам пользователю

In [ ]:
def ruleToText(rule, delta_dr):
    result_text='Рекомендуется ограничить выдачи по заявкам, где '
    for r in rule[:-1]:
        result_text+='{} {} {} и '.format(r[0], r[1], round(r[2],4))
    result_text = result_text[:-3] + '. Что позволит снизить DR на {}%'.format(round(delta_dr*100, 2))
    return result_text
    
ruleToText(best_multirule, dr_delta)